# Depth Image Transformations

In [1]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd

path = 'D:/Desktop/ransac_obstacle_detection/saved' #set to your desired path to save images in
frames_dir = "test_frames_straight/" #the directory holding the saved depth images in .npy format

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -30, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    """
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)

    Given a depth image, converts each
    point to an XYZ coordinate and returns
    an array of these points.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    """
    x, y, z = (row, col, depth)
    
    Given a point from a depth image
    and its position in the image,
    returns the x, y, and z coordinates
    relative to the camera location.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # This runs slowly in Python as it is required to be called within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # use trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[ax1] ** 2 + pt[ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[ax2], pt[ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(0, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(1, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y plane

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def reject_outliers(data, m=3):
    return data[np.abs(np.subtract(data, np.mean(data, axis=0))) < m * np.std(data, axis=0)]


# Plane Fitting

In [6]:
def plane_fit(points):
    """
    p, n = plane_fit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_orientation(xyz_arr, num_points, num_planes):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, num_points):
        rand_points = []
        while len(rand_points) < 10:
            index = random.randrange(0,len(xyz_arr))
            if not xyz_arr[index].all() == np.zeros(3).all():
                rand_points.append(xyz_arr[index])
            #print(np.array(rand_points))
        rand_points = np.array(rand_points).T
        for _ in range(0, num_planes):
            ctr, P = plane_fit(rand_points)
            r = math.sqrt(P[0]**2 + P[1]**2 + P[2]**2)
            theta = math.acos(P[2]/r) * 180 / math.pi
            phi = math.atan(P[1]/P[0]) * 180 / math.pi
            pitch.append(theta)
            planes.append(P)
            centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    
    
from sklearn.linear_model import RANSACRegressor
from sklearn.datasets import make_regression

In [3]:
# Inital guess of the plane
p0 = [0.506645455682, -0.185724560275, -1.43998120646, 1.37626378129]

def f_min(X,p):
    plane_xyz = p[0:3]
    #print('plane_xyz', plane_xyz)
    distance = (plane_xyz*X).sum(axis=1) + p[3]
    return distance / np.linalg.norm(plane_xyz)

def residuals(params, signal, X):
    return f_min(X, params)

from scipy.optimize import leastsq

def fit_plane_optimize(arr):
    #arr = reject_outliers(arr)
    arr = np.random.permutation(arr)
    arr = np.reshape(arr, (np.shape(arr)[0], -1))
    ctr = arr.mean(axis=1)
    sol = leastsq(residuals, p0, args=(None, arr))[0]
    plane_xyz = sol[0:3]
    r = math.sqrt(plane_xyz[0]**2 + plane_xyz[1]**2 + plane_xyz[2]**2)
    theta = math.acos(plane_xyz[2]/r) * 180 / math.pi
    phi = math.atan(plane_xyz[1]/plane_xyz[0]) * 180 / math.pi
    return ctr[0:3], plane_xyz, phi

def get_orientation(xyz_arr, num_planes, num_points):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, num_planes):
        rand_points = np.random.permutation(xyz_arr.T[np.all(xyz_arr.T, axis=1)])[:num_points]
        #for point in rand_points:
        #    print('Point', point)
        #for point_t in rand_points.T:
        #    print('Point.T', point_t)
        #print('Random points:\n', rand_points)
        #print('Transpose:\n', rand_points.T)
        ctr, P, phi = plane_fit(rand_points)
        pitch.append(phi)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    

# Obstacle Detection

In [4]:
import numpy as np
import math
import random
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyntcloud import PyntCloud
from IPython.display import display, HTML

def get_obstacles_with_plane(depth_frame, num_planes, num_points, dist_thresh, offset, visualize):
    obstacles = np.zeros(depth_frame.shape) #empty image that will store the locations of detected obstacles
    img = np.uint8(depth_frame) #some opencv functions require a byte image
    kernel = np.ones((3,3),np.uint8)
    _, foreground = cv2.threshold(depth_frame,0.5,1.5, cv2.THRESH_TOZERO) 
    cropped = np.zeros(depth_frame.shape, np.float32)
    cropped[depth_frame.shape[0]//4:] = depth_frame[depth_frame.shape[0]//4:]
    xyz_arr = depthMatrixToPointCloudPos(cropped) #convert depth data to XYZ coordinates
    #print(fit_plane_optimize(xyz_arr.T))
    start_time = time.time()
    center, plane, theta = get_orientation(xyz_arr, num_planes, num_points)
    #center, plane, theta = fit_plane_optimize(reject_outliers(xyz_arr))
    print(center, plane, theta)
    CameraPosition['elevation'] = -theta
    center = applyCameraOrientation(center)
    plane = applyCameraOrientation(plane)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    plane_img = np.zeros(len(xyz_arr))
    plane_img[xyz_arr[:,2] > dist_thresh - center[2] - offset] = 1
    if visualize:
        #xyz_arr = xyz_arr[xyz_arr[:,2] > dist_thresh - center[2]]
        points = pd.DataFrame(xyz_arr, columns=['x', 'y', 'z']) #convert XYZ coordinates to a DataFrame for pyntcloud
        cloud = PyntCloud(points)

        lines = [
            {
                # X axis
                "color": "red", 
                "vertices": [[0, 0, 0], [10, 0, 0]]
            },
            {
                # Y axis
                "color": "green",
                "vertices": [[0, 0, 0], [0, 10, 0]]
            },
            {
                # Z axis
                "color": "blue",
                "vertices": [[0, 0, 0], [0, 0, 10]]
            },
            {
                #Original norm of the plane
                "color": "pink",
                "vertices": [center.tolist(), plane.tolist()]
            }
        ]
        cloud.plot(cmap="cool", polylines=lines)

    plane_img = np.uint8(np.reshape(plane_img,(424,512)) * 255) #reshape to match depth data and convert to uint8
    plane_img = np.uint8((np.ones((424,512)) * 255) - plane_img) #invert img so pixel value corresponds to NOT ground plane
    ret, plane_img = cv2.threshold(plane_img,0,255,cv2.THRESH_BINARY) #filter points that are probaly not ground plane
    plane_img = cv2.subtract(img, plane_img)
    
    #noise removal
    kernel = np.ones((3,3),np.uint8)
    #opening = cv2.morphologyEx(plane_img ,cv2.MORPH_OPEN, kernel, iterations = 6) #erosion followed by dilation
    erode = cv2.erode(plane_img, kernel, iterations=4)
    _, thresh = cv2.threshold(erode,0,255,cv2.THRESH_BINARY)
    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #BGR image to draw labels on

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent circle
            #this measurement is only used for checking if countours fit our bounds
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)
            
            x,y,obj_length,obj_height = cv2.boundingRect(cntr)

            equi_diameter = obj_length #bounding rectangle gives a better approximation of diameter
            
            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 200
            
            HIGH_MM_BOUND = 400

            HIGH_DISTANCE_BOUND = 6000
            #Original tolerances were 20 and 150

            if(equi_diameter > LOW_DIAMETER_BOUND and equi_diameter < HIGH_DIAMETER_BOUND):
                mask = np.zeros_like(img) #mask will contain the fitted and adjusted ellipse of a single obstacle
                ellipse = cv2.fitEllipse(cntr)
                rect = cv2.minAreaRect(cntr)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1) #draw the fitted ellipse
                rows = mask.shape[0]
                cols = mask.shape[1]
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]]) #shift mask down to match obstacle, not edge
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3) #erode the mask to remove background points
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask) #use the mask to isolate original depth values
                img_fg = cv2.medianBlur(img_fg,5) #median blur to further remove noise
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles)) 
                
                mean_val = np.median(img_fg[img_fg.nonzero()]) #compute the non-zero average of obstacle depth values
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)
                mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val #convert pixel diameter to mm


                moment = cv2.moments(cntr) #get the centroid of the obstacle using its moment
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND and mm_diameter < HIGH_MM_BOUND: #kinect loses accuracy beyond 4.5m
                    coords = depthToPointCloudPos(cx, cy, mean_val) #convert obstacle depth to XYZ coordinate
                    #begin visualization
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(color, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(color, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to process image")
            print (sys.exc_info()[0])
            
    elapsed_time = time.time() - start_time
    print("Frame took " + str(elapsed_time) + " seconds to process")

    cv2.imshow("plane",plane_img)
    cv2.imshow("proc", thresh)
    cv2.imshow("img",depth_frame/4500.0)
    cv2.imshow("final",color)
    cv2.imshow("denoised",cropped/4500.)
    cv2.imshow("obstacles",obstacles)
    key = cv2.waitKey(delay=1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        sys.exit(0)
    return color



# Single Image Testing

In [7]:
img = np.load('test_frames_arc/200.npy')
output = get_obstacles_with_plane(img, num_planes = 18, 
                                  num_points = 16, 
                                  dist_thresh = 0.1, 
                                  offset = 0.175,
                                  visualize = True)

[ 1.01611449 -0.06722823  0.07608881] [-0.28104612 -0.03285124  0.95888547] 16.45319243590497


C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\pythreejs\traits.py:175: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.29614631472927894, 1.4095932024611653, -0.…

Frame took 0.7008657455444336 seconds to process


# Multiple Frame Testing

In [13]:
frames_dir = 'test_frames_straight/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.load(file_path)
        try:
            output = get_obstacles_with_plane(img, num_planes =26, 
                                  num_points = 25, 
                                  dist_thresh = 0.07,
                                  offset = 0.175,
                                  visualize = False)
            cv2.imwrite(os.path.join(path ,filename[:-4] + '.png'), output)
            frame_i += 1
        except:
            print("Could not find obstacles")
cv2.destroyAllWindows()

C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  after removing the cwd from sys.path.


test_frames_straight/0.npy
[ 1.49809712  0.0156126  -0.31664944] [-0.15230479  0.03058719  0.98763386] 8.982355812402762
Frame took 0.10527968406677246 seconds to process
test_frames_straight/1.npy
[ 1.50101476 -0.05997339 -0.31896301] [-0.13991709  0.019338    0.98979134] 8.166960749847828
Frame took 0.13335824012756348 seconds to process
test_frames_straight/10.npy
[ 1.39389953 -0.03338298 -0.31996438] [-0.13056311  0.07921861  0.9880214 ] 8.85176375797147
Frame took 0.12332653999328613 seconds to process
test_frames_straight/100.npy
[1.11774573 0.01994335 0.02678105] [-0.27710633  0.04120026  0.95970348] 16.294341810160148
Frame took 0.11530232429504395 seconds to process
test_frames_straight/101.npy
[ 1.03006613 -0.02832513  0.00440893] [-0.28045233  0.04310594  0.95855949] 16.518884246403456
Frame took 0.12934494018554688 seconds to process
test_frames_straight/102.npy
[ 1.07799863 -0.04162797  0.01816   ] [-0.27322293  0.04068342  0.96098636] 16.050895767186052
Frame took 0.12633

test_frames_straight/141.npy
[ 1.10897946 -0.04191789  0.02997916] [-0.25474057  0.05131428  0.96543735] 15.097186498152508
Frame took 0.12232255935668945 seconds to process
test_frames_straight/142.npy
[1.13307607 0.0044296  0.04692447] [-0.26552542  0.04779512  0.96258387] 15.702467830028473
Frame took 0.11731243133544922 seconds to process
test_frames_straight/143.npy
[ 1.06062407 -0.0295725   0.0411516 ] [-0.27273156  0.05497234  0.9602614 ] 16.192466211323122
Frame took 0.12431001663208008 seconds to process
test_frames_straight/144.npy
[ 1.1005883  -0.00995352  0.06291413] [-0.2795208   0.05281312  0.95837717] 16.573975001683895
Frame took 0.11831331253051758 seconds to process
test_frames_straight/145.npy
[ 0.9897465  -0.01474518  0.04231498] [-0.29349757  0.05208027  0.95411436] 17.38581143923246
Frame took 0.12232589721679688 seconds to process
test_frames_straight/146.npy
[1.07144549 0.01144412 0.0795117 ] [-0.29788518  0.04281014  0.9532137 ] 17.583355725857707
Frame took 0.

C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in less


test_frames_straight/148.npy
[ 1.14991763 -0.04014907  0.12898531] [-0.30988201  0.03524273  0.94983647] 18.213449632221778
Frame took 0.17246079444885254 seconds to process
test_frames_straight/149.npy
[ 1.07686885 -0.00769948  0.11117979] [-0.30760034  0.03023029  0.95084867] 18.029132832466395
Frame took 0.16343474388122559 seconds to process
test_frames_straight/15.npy
[ 1.36342237 -0.0438041  -0.27621855] [-0.15343404  0.0672321   0.98567372] 9.695599677878688
Frame took 0.11930465698242188 seconds to process
test_frames_straight/150.npy
[1.02969608 0.03651234 0.1085176 ] [-0.31843747  0.01208914  0.9475182 ] 18.63201773748094
Frame took 0.13234972953796387 seconds to process
test_frames_straight/151.npy
[1.00346451 0.00825977 0.1057103 ] [-0.32102602  0.01718872  0.94683256] 18.761277072532817
Frame took 0.1353600025177002 seconds to process
test_frames_straight/152.npy
[1.04427551 0.03177032 0.12485654] [-0.32820845  0.01356763  0.94434005] 19.200554883495048
Frame took 0.148395

...

Frame took 0.12131190299987793 seconds to process
test_frames_straight/363.npy
[ 1.2374523  -0.04588913 -0.40552194] [0.02942916 0.06191207 0.99346128] 5.639938580024767
Frame took 0.12733912467956543 seconds to process
test_frames_straight/364.npy
[ 1.23691504 -0.06433943 -0.41039467] [0.03259255 0.07322562 0.99662552] 4.665398058517283
Frame took 0.12629246711730957 seconds to process
test_frames_straight/365.npy
[ 1.25731261 -0.03988229 -0.40958632] [0.0229338  0.06173844 0.99709121] 4.225852661293989
Frame took 0.11931824684143066 seconds to process
test_frames_straight/366.npy
[ 1.25628784 -0.10329973 -0.40958814] [0.03052424 0.07391435 0.99654914] 4.650328142281655
Frame took 0.13135266304016113 seconds to process
test_frames_straight/367.npy
[ 1.28733697e+00  1.08842271e-03 -4.19506152e-01] [0.03389996 0.07717021 0.99585706] 5.13003685236572
Frame took 0.12232446670532227 seconds to process
test_frames_straight/368.npy
[ 1.25004557 -0.11089886 -0.40808142] [0.0284004  0.06744074

[ 1.19375549 -0.04955583 -0.270278  ] [-0.12263733  0.08836932  0.9877253 ] 8.860068655941387
Frame took 0.12633466720581055 seconds to process
test_frames_straight/407.npy
[ 1.18991544 -0.03092494 -0.26212407] [-0.12609396  0.09265508  0.98717062] 9.086774612550668
Frame took 0.13335323333740234 seconds to process
test_frames_straight/408.npy
[ 1.18487348 -0.07280852 -0.2500881 ] [-0.14023546  0.09206997  0.98516383] 9.785653857490644
Frame took 0.1213231086730957 seconds to process
test_frames_straight/409.npy
[ 1.19888421 -0.01592604 -0.26122819] [-0.13101701  0.09613004  0.98648095] 9.406691624616663
Frame took 0.1072845458984375 seconds to process
test_frames_straight/41.npy
[ 1.16850302 -0.02417263 -0.15046417] [-0.22106339  0.08479819  0.97122343] 13.753083303136679
Frame took 0.1072845458984375 seconds to process
test_frames_straight/410.npy
[ 1.23738817 -0.11620766 -0.2573269 ] [-0.11502283  0.09890344  0.98832137] 8.74124579555813
Frame took 0.12232279777526855 seconds to pro

[ 1.16941071  0.00304583 -0.11431187] [-0.21449432  0.09279228  0.97213754] 13.541730338634004
Frame took 0.12734508514404297 seconds to process
test_frames_straight/450.npy
[ 1.27980333 -0.12234635 -0.2563829 ] [-0.13552818  0.12676592  0.98246408] 10.722902323874948
Frame took 0.1483914852142334 seconds to process
test_frames_straight/451.npy
[ 1.23817359 -0.13048768 -0.26875377] [-0.12857526  0.12609619  0.98358724] 10.387051292932979
Frame took 0.13736200332641602 seconds to process
test_frames_straight/452.npy
[ 1.25534914 -0.06024236 -0.27960503] [-0.13185579  0.13032335  0.98250872] 10.709603685940367
Frame took 0.12433362007141113 seconds to process
test_frames_straight/453.npy
[ 1.25719387 -0.08962918 -0.28173244] [-0.12457191  0.13938049  0.98224324] 10.79617813457002
Frame took 0.13937139511108398 seconds to process
test_frames_straight/454.npy
[ 1.22632841  0.0065257  -0.30124618] [-0.12278071  0.14580537  0.98154365] 11.007584140936226
Frame took 0.13936829566955566 second

test_frames_straight/493.npy
[ 1.22682553 -0.06063076 -0.28128613] [-0.14981295  0.03831787  0.98774369] 8.9424043804343
Frame took 0.13234972953796387 seconds to process
test_frames_straight/494.npy
[ 1.29071351 -0.05388128 -0.26713815] [-0.15134018  0.04204215  0.98747734] 9.062088594074579
Frame took 0.12031865119934082 seconds to process
test_frames_straight/495.npy
[ 1.25986316 -0.05414024 -0.25699819] [-0.15805542  0.04550169  0.98626228] 9.485160793018094
Frame took 0.14137482643127441 seconds to process
test_frames_straight/496.npy
[ 1.1762884  -0.02534561 -0.25944357] [-0.17012569  0.03924748  0.98454424] 10.07518940592699
Frame took 0.1413745880126953 seconds to process
test_frames_straight/497.npy
[ 1.10333497 -0.00754111 -0.26229013] [-0.17875136  0.03899342  0.98291221] 10.565463199774086
Frame took 0.13938570022583008 seconds to process
test_frames_straight/498.npy
[ 1.21688819 -0.04987979 -0.23457079] [-0.18045886  0.02971992  0.9829552 ] 10.547553320736288
Frame took 0.

Frame took 0.13135051727294922 seconds to process
test_frames_straight/87.npy
[1.09015909 0.05935405 0.09053721] [-0.34014869  0.06085465  0.93810108] 20.24815362007061
Frame took 0.13135099411010742 seconds to process
test_frames_straight/88.npy
[0.99805817 0.00977968 0.05651792] [-0.33808759  0.06275589  0.93860123] 20.16338557230003
Frame took 0.1413729190826416 seconds to process
test_frames_straight/89.npy
[1.00126442 0.05392435 0.04802573] [-0.33213694  0.06943895  0.94043455] 19.85553653103295
Frame took 0.12232613563537598 seconds to process
test_frames_straight/9.npy
[ 1.34446489 -0.00404812 -0.33142291] [-0.13106751  0.07124825  0.98851067] 8.636158404888256
Frame took 0.12432503700256348 seconds to process
test_frames_straight/90.npy
[ 1.07251943 -0.01612262  0.07326452] [-0.32699707  0.0603329   0.94279458] 19.450581176955176
Frame took 0.13435745239257812 seconds to process
test_frames_straight/91.npy
[1.06569774 0.01468806 0.0617645 ] [-0.31863345  0.06211336  0.94563559]